In [1]:
%reload_ext autoreload
%autoreload 2


In [2]:
import sys
sys.path.insert(0, r'/Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/build')

import numpy as np

import cpp_probabilities

## Load data

In [3]:
from result_saver import SaverProvider
provider = SaverProvider()
backend = provider.get_backend('ibmq_mumbai') 
job = provider.retrieve_job("cmyhbrqrmwhg008bs4h0") # Mumbai job
# job = provider.retrieve_job("cn6g47862r90008810pg") # Sherbrooke job
memory = job.result().get_memory()

print(memory.shape)


(1000, 9)


In [6]:
from soft_info import get_KDEs, get_repcode_layout, get_repcode_IQ_map

## Mumbai
layout = [25, 19, 26, 22, 16] # To implement: into scratch job_data
kde_dict, scaler_dict = get_KDEs(provider, 'ibmq_mumbai', list(range(27)), bandwidths=0.2, plot=False)
synd_rounds = 3

## Sherbrooke
# layout = get_repcode_layout(30, provider.get_backend('ibm_sherbrooke') )
# kde_dict, scaler_dict = get_KDEs(provider, 'ibm_sherbrooke', list(range(127)), bandwidths=0.2, plot=False)
# synd_rounds = 35

print("Generating the qubit mapping...")
qubit_mapping = get_repcode_IQ_map(layout, synd_rounds) #Hardcoded for repetition codes

Generating the qubit mapping...


# Rescale cpp

In [7]:
# GENERATE GRID DATA

import numpy as np

grid_dict = {}
num_points = 10
for qubit_idx, (kde_0, kde_1) in kde_dict.items():
    scaler = scaler_dict[qubit_idx]

    # Retrieve the dataset for this qubit and split into real and imaginary parts
    data = np.array(memory[0])  # TODO change that: Retrieving data from memory 0 to get the min and max values
    data_real_imag = np.column_stack([np.real(data), np.imag(data)])

    # Scale data
    scaled_data = scaler.transform(data_real_imag)

    # Create grid
    grid_x, grid_y = np.linspace(scaled_data[:, 0].min() - 1, scaled_data[:, 0].max() + 1, num_points), \
                     np.linspace(scaled_data[:, 1].min() - 1, scaled_data[:, 1].max() + 1, num_points)
    grid_x, grid_y = np.meshgrid(grid_x, grid_y)
    grid_points = np.vstack([grid_x.ravel(), grid_y.ravel()]).T

    # Evaluate KDE on the grid for both states
    grid_density_0 = (kde_0.score_samples(grid_points)).reshape(grid_x.shape)
    grid_density_1 = (kde_1.score_samples(grid_points)).reshape(grid_x.shape)
    
    # Create an instance of GridData and store in dictionary
    grid_dict[qubit_idx] = cpp_probabilities.GridData(grid_x, grid_y, grid_density_0, grid_density_1)

In [8]:
from cpp import process_scaler_dict


processed_scaler_dict = process_scaler_dict(scaler_dict)

counts = cpp_probabilities.get_counts(memory, qubit_mapping, grid_dict, processed_scaler_dict, synd_rounds)
(sorted(counts.items(), key=lambda x: x[1], reverse=True))

[('000 00 00 00', 857),
 ('000 01 00 00', 18),
 ('000 00 01 00', 11),
 ('000 01 01 01', 11),
 ('000 10 10 10', 11),
 ('001 00 00 00', 8),
 ('100 00 10 00', 8),
 ('100 10 00 10', 7),
 ('000 10 00 00', 6),
 ('000 10 10 00', 6),
 ('001 01 00 01', 5),
 ('010 11 00 11', 5),
 ('000 00 00 01', 4),
 ('001 00 01 00', 4),
 ('001 01 00 00', 4),
 ('100 10 00 00', 4),
 ('000 00 10 00', 3),
 ('010 00 00 00', 3),
 ('100 00 00 00', 3),
 ('000 00 10 10', 2),
 ('000 01 01 00', 2),
 ('000 10 00 10', 2),
 ('000 00 00 10', 1),
 ('000 11 01 01', 1),
 ('000 11 11 10', 1),
 ('001 01 01 00', 1),
 ('001 01 01 01', 1),
 ('001 10 10 00', 1),
 ('001 11 10 10', 1),
 ('010 00 11 00', 1),
 ('010 01 00 11', 1),
 ('010 01 10 01', 1),
 ('010 10 00 00', 1),
 ('010 10 01 00', 1),
 ('010 11 00 00', 1),
 ('010 11 01 00', 1),
 ('100 00 10 01', 1),
 ('100 10 10 00', 1)]